# Экспертная система - Еда

В этом упражнении мы реализуем экспертную систему "животные" из одной классической книги. Дерево И-ИЛИ системы приведено на рисунке ниже:

## Обратный вывод



In [8]:
class Ask():
    def __init__(self, choices=['y', 'n']):
        self.choices = choices

    def ask(self):
        if max([len(x) for x in self.choices]) > 1:
            for i, x in enumerate(self.choices):
                print("{0}. {1}".format(i, x))
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices))
            return input()


class Content():
    def __init__(self, x):
        self.x = x


class If(Content):
    pass


class AND(Content):
    pass


class OR(Content):
    pass

In [9]:
rules = {
    'default': Ask(['y', 'n']),
    'вид еды': Ask(['основное блюдо', 'закуска', 'десерт']),
    'тип блюда': Ask(['мясное', 'вегетарианское', 'морепродукты']),
    'время суток': Ask(['утро', 'день', 'вечер']),
    'предпочтение': Ask(['здоровое', 'калорийное']),
    'не калорийное': If(OR(['предпочтение:здоровое'])),
    'завтрак': If(AND(['время суток:утро'])),
    'обед': If(AND(['время суток:день'])),
    'ужин': If(AND(['время суток:вечер'])),
    'для вегетарианцев': If(AND(['тип блюда:вегетарианское'])),
    'десерт': If(AND(['вид еды:десерт'])),
    'еда:овсянка': If(['тип блюда:вегетарианское', 'завтрак']),
    'еда:салат': If(['тип блюда:вегетарианское', 'не калорийное', 'обед']),
    'еда:стейк': If(['тип блюда:мясное', 'калорийное', 'ужин']),
    'еда:рыба': If(['тип блюда:морепродукты', 'ужин']),
    'еда:паста': If(['тип блюда:вегетарианское', 'калорийное', 'ужин']),
    'еда:шоколад': If(['десерт', 'предпочтение:калорийное']),
    'еда:фрукты': If(['десерт', 'предпочтение:здоровое']),
}

In [10]:
class KnowledgeBase():
    def __init__(self, rules):
        self.rules = rules
        self.memory = {}

    def get(self, name):
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld == name or fld.startswith(name + ":"):
                value = 'y' if fld == name else fld.split(':')[1]
                res = self.eval(self.rules[fld], field=name)
                if res == 'y':
                    self.memory[name] = value
                    return value
        res = self.eval(self.rules['default'], field=name)
        self.memory[name] = res
        return res

    def eval(self, expr, field=None):
        if isinstance(expr, Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr, If):
            return self.eval(expr.x)
        elif isinstance(expr, AND) or isinstance(expr, list):
            expr = expr.x if isinstance(expr, AND) else expr
            for x in expr:
                if self.eval(x) == 'n':
                    return 'n'
            return 'y'
        elif isinstance(expr, OR):
            for x in expr.x:
                if self.eval(x) == 'y':
                    return 'y'
            return 'n'
        elif isinstance(expr, str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

In [12]:
kb = KnowledgeBase(rules)
kb.get('еда')

тип блюда:вегетарианское
y/n
тип блюда:мясное
y/n
калорийное
y/n
время суток:вечер
y/n
ужин
y/n
тип блюда:морепродукты
y/n
вид еды:десерт
y/n
предпочтение:калорийное
y/n


'шоколад'

## Прямой вывод

Прямой вывод мы реализуем на базе библиотеки [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** - это библиотека для построения экспертных систем прямого вывода на Python, которая спроектирована так, чтобы быть похожей на классическую систему [CLIPS](http://www.clipsrules.net/index.html). 

In [1]:
from pyknow import *

In [2]:
class Food(KnowledgeEngine):

    @Rule(OR(Fact('утро'), Fact('день'), Fact('вечер')))
    def set_time(self):
        self.declare(Fact('время суток'))

    @Rule(AND(Fact('утро')))
    def breakfast(self):
        self.declare(Fact('завтрак'))

    @Rule(AND(Fact('день')))
    def lunch(self):
        self.declare(Fact('обед'))

    @Rule(AND(Fact('вечер')))
    def dinner(self):
        self.declare(Fact('ужин'))

    @Rule(AND(Fact('вегетарианское'), Fact('завтрак')))
    def oats(self):
        self.declare(Fact(thing='овсянка'))

    @Rule(AND(Fact('вегетарианское'), Fact('обед'), Fact('не калорийное')))
    def salad(self):
        self.declare(Fact(thing='салат'))

    @Rule(AND(Fact('мясное'), Fact('ужин'), Fact('калорийное')))
    def steak(self):
        self.declare(Fact(thing='стейк'))

    @Rule(AND(Fact('морепродукты'), Fact('ужин')))
    def fish(self):
        self.declare(Fact(thing='рыба'))

    @Rule(AND(Fact('вегетарианское'), Fact('ужин'), Fact('калорийное')))
    def pasta(self):
        self.declare(Fact(thing='паста'))

    @Rule(AND(Fact('десерт'), Fact('калорийное')))
    def chocolate(self):
        self.declare(Fact(thing='шоколад'))

    @Rule(AND(Fact('десерт'), Fact('здоровое')))
    def fruits(self):
        self.declare(Fact(thing='фрукты'))

    @Rule(Fact(thing=MATCH.a))
    def print_result(self, a):
        print('Еда - {}'.format(a))

    def factz(self, l):
        for x in l:
            self.declare(x)

In [3]:
ex1 = Food()
ex1.reset()
ex1.factz([
    Fact('утро'),
    Fact('вегетарианское')])
ex1.run()
ex1.facts

Еда - овсянка


FactList([(0, InitialFact()),
          (1, Fact('утро')),
          (2, Fact('вегетарианское')),
          (3, Fact('время суток')),
          (4, Fact('завтрак')),
          (5, Fact(thing='овсянка'))])

Еда - овсянка
FactList([(0, InitialFact()),
          (1, Fact('утро')),
          (2, Fact('вегетарианское')),
          (3, Fact('время суток')),
          (4, Fact('завтрак')),
          (5, Fact(thing='овсянка'))])